In [53]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy import signal
import glob
from sklearn.metrics import mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy import signal
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy import signal
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import mlflow.pyfunc.model



In [54]:
csv_files = glob.glob(
    r"C:\Users\Max_G\OneDrive\IUBH\5. Semester\Model Engeneering\Public_Transport_Forecasting\data\interim\Cluster_Data_All_Month\*.csv"
)

In [55]:
weekday_file_paths = [path for path in csv_files if 'weekday' in path]
weekend_file_paths = [path for path in csv_files if 'weekend' in path]

In [56]:
weekday_dict_all_data = {}

for index, file in enumerate(weekday_file_paths):
    current_df = pd.read_csv(file, index_col=[0])
    current_df['Date'] =  pd.to_datetime(current_df['Date'])
    current_df.index = pd.to_datetime(current_df.index)

    weekday_dict_all_data[int(weekday_file_paths[index][-13:-12])] = current_df

In [57]:
endog_week_dict_all_data = {}

for label in weekday_dict_all_data:
    current_df = weekday_dict_all_data[label]
    current_df = current_df.reset_index()
    current_df = current_df.drop(columns=['index', 'Hour', 'Weekday', 'Date'])
    endog_week_dict_all_data[label] = current_df

In [58]:
weekday_dict_april = {}

for index, file in enumerate(weekday_file_paths):
    current_df = pd.read_csv(file, index_col=[0])
    current_df['Date'] =  pd.to_datetime(current_df['Date'])
    current_df.index = pd.to_datetime(current_df.index)
    current_df = current_df[(current_df['Date'].dt.month == 4)]

    weekday_dict_april[int(weekday_file_paths[index][-13:-12])] = current_df

In [59]:
endog_week_dict_april = {}

for label in weekday_dict_april:
    current_df = weekday_dict_april[label]
    current_df = current_df.reset_index()
    current_df = current_df.drop(columns=['index', 'Hour', 'Weekday', 'Date'])
    endog_week_dict_april[label] = current_df

In [60]:
def getMaxPeriodogram(freq, Pxx_spec, num_top=5):
    # Find the indices of the top 'num_top' maximum PSDs
    top_indices = np.argsort(Pxx_spec)[-num_top:][::-1]

    # Extract the frequencies and PSDs for the top 'num_top' maximum PSDs
    freqMax = freq[top_indices]
    maxPsd = Pxx_spec[top_indices]
    # Convert to periods
    periodMax = 1 / freqMax
    return maxPsd, freqMax, periodMax


In [61]:
tracking_uri = "file:///C:/Users/Max_G/OneDrive/IUBH/5. Semester/Model Engeneering/Public_Transport_Forecasting/models/Tracking"
mlflow.set_tracking_uri(tracking_uri)
# Create a named experiment
mlflow.set_experiment("predicting_per_cluster_april_new_1")
experiment = mlflow.get_experiment_by_name("predicting_per_cluster_april_new_1")

In [63]:
def sarima(data, label, order, seasonal_order):
    
    # Fit the SARIMA model with harmonic regression (exogenous variables)
    with mlflow.start_run(experiment_id=experiment.experiment_id, run_name=f"SARIMA{order}, {seasonal_order}, {label}") as run:
        
        # Fit the SARIMA model with exogenous variables
        model = SARIMAX(data[:-48], order=order, seasonal_order=seasonal_order)
        results = model.fit()
        
        # Calculate AIC, RMSE for training, and RMSE for testing
        aic = results.aic
        y_pred_train = results.fittedvalues
        y_pred_test = results.forecast(steps=48)
        rmse_train = np.sqrt(mean_squared_error(data[:-48], y_pred_train))
        rmse_test = np.sqrt(mean_squared_error(data[-48:], y_pred_test))
        mae_train = mean_absolute_error(data['Count'][:-48], y_pred_train)
        mae_test = mean_absolute_error(data['Count'][-48:], y_pred_test)
        train_size = len(data[:-48])
        test_size = len(data['Count'][-48:])

        # Log AIC and R-squared
        mlflow.log_metrics({
            "AIC": aic,
            "RMSE_Train" : rmse_train,
            "RMSE_Test" : rmse_test,
            "MAE_Train" : mae_train,
            "MAE_Test" : mae_test,
            "MAE_Train" : mae_train,
            "MAE_Test" : mae_test,
            "MAE_Train_Percentage" : mae_train/train_size,
            "MAE_Test_Percentage" : mae_test/test_size

        })

        mlflow.log_params({
        "order": order,
        "seasonal_order": seasonal_order,
        'label' : label,
        'test_size' : test_size,
        'train_size' : train_size
        })
        # Log the trained model
        mlflow.sklearn.log_model(results, "SARIMA")

In [64]:
def return_periodicities(data):
    freq, Pxx_spec = signal.periodogram(data.Count, scaling="spectrum")
    maxPsd, freqMax, periodMax = getMaxPeriodogram(freq, Pxx_spec)
        
    # Define periods
    period01, period02, period03, period04, period05 = list(1 / freqMax)

    # Extract the number hourly samples
    T = len(data)
    time = np.arange(1, T + 1)

    # Create regressors for the daily period
    omega01 = 2 * np.pi / period01
    s1 = np.cos(omega01 * time)
    c1 = np.sin(omega01 * time)

    # Create regressors for the weekly period
    omega02 = 2 * np.pi / period02
    s2 = np.cos(omega02 * time)
    c2 = np.sin(omega02 * time)

    omega03 = 2 * np.pi / period03
    s3 = np.cos(omega03 * time)
    c3 = np.sin(omega03 * time)

    omega04 = 2 * np.pi / period04
    s4 = np.cos(omega04 * time)
    c4 = np.sin(omega04 * time)

    omega05 = 2 * np.pi / period05
    s5 = np.cos(omega05 * time)
    c5 = np.sin(omega05 * time)

    # Stack regressors and add intercept
    regressors = np.column_stack((s1, c1, s2, c2, s3, c3, s4, c4, s5, c5))
    regressors_one = sm.add_constant(regressors)

    exog_var_week = regressors    

    return regressors_one, exog_var_week

In [65]:
def pure_harmonic_regression(data, label):  

    regressors_one, exog_var_week = return_periodicities(data) 
    
# Create an MLflow experiment and start a run
    with mlflow.start_run(experiment_id=experiment.experiment_id, run_name=f"april_harmonic_week_{label}") as run:
        

        # Create and fit the regression model
        model = sm.OLS(data[:-48], regressors_one[:-48])
        results_harmonic_week = model.fit()

        # Make predictions
        y_pred_train = results_harmonic_week.predict(regressors_one[:-48])
        y_pred_test = results_harmonic_week.predict(regressors_one[-48:])

        # Calculate AIC and R-squared
        aic = results_harmonic_week.aic
        r_squared = results_harmonic_week.rsquared
        rmse_train = np.sqrt(mean_squared_error(data[:-48], y_pred_train))
        rmse_test = np.sqrt(mean_squared_error(data[-48:], y_pred_test))
        mae_train = mean_absolute_error(data['Count'][:-48], y_pred_train)
        mae_test = mean_absolute_error(data['Count'][-48:], y_pred_test)
        train_size = len(data[:-48])
        test_size = len(data['Count'][-48:])


        # Log model parameters
        mlflow.log_params({
            'label' : label,
            'test_size' : test_size,
            'train_size' : train_size
        })

        # Log AIC and R-squared
        mlflow.log_metrics({
            "AIC": aic,
            "R-squared": r_squared,
            "RMSE_Train" : rmse_train,
            "RMSE_Test" : rmse_test,
            "MAE_Train" : mae_train,
            "MAE_Test" : mae_test,
            "MAE_Train_Percentage" : mae_train/train_size,
            "MAE_Test_Percentage" : mae_test/test_size

        })

        # Log the trained model
        mlflow.sklearn.log_model(results_harmonic_week, "harmonic_model")

        
        

In [66]:
def sarimax(data, label): 

    regressors_one, exog_var_week = return_periodicities(data) 

    # Create an MLflow experiment and start a run
    with mlflow.start_run(experiment_id=experiment.experiment_id, run_name=f"SARIMAX {label}") as run:
        order = (24, 0, 0)
        seasonal_order = (0, 0, 0, 0)
        model_sarimax = SARIMAX(
            endog= data[:-48],
            exog=exog_var_week[:-48],
            trend="c",
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=True,
            enforce_invertibility=True,
        )
        m_sarimax_24 = model_sarimax.fit()

        # Make predictions
        y_pred_train = m_sarimax_24.predict()
        forecast = m_sarimax_24.get_forecast(steps=48, exog=exog_var_week[-48:])
        y_pred_test = forecast.predicted_mean

        # Calculate AIC and R-squared
        aic = m_sarimax_24.aic
        
        rmse_train = np.sqrt(mean_squared_error(data['Count'][:-48], y_pred_train))
        rmse_test = np.sqrt(mean_squared_error(data['Count'][-48:], y_pred_test))
        mae_train = mean_absolute_error(data['Count'][:-48], y_pred_train)
        mae_test = mean_absolute_error(data['Count'][-48:], y_pred_test)
        train_size = len(data[:-48])
        test_size = len(data['Count'][-48:])

        # Log AIC and R-squared
        mlflow.log_metrics({
            "AIC": aic,
            "RMSE_Train" : rmse_train,
            "RMSE_Test" : rmse_test,
            "MAE_Train" : mae_train,
            "MAE_Test" : mae_test,  
            "MAE_Train_Percentage" : mae_train/train_size,
            "MAE_Test_Percentage" : mae_test/test_size
        })

        # Log model parameters
        mlflow.log_params({
            'label' : label,
            'test_size' : test_size,
            'train_size' : train_size
        })

        # Log the trained model
        mlflow.sklearn.log_model(m_sarimax_24, "SARIMAX")

In [67]:
def function_wrapper(data, label):
    print('started sarima', label)
    sarima(data, label, (0, 1, 6), (1, 1, 1, 24))
    print('started sarima', label)
    sarima(data, label, (1, 0, 0), (1, 1, 0, 24))
    print('started sarimax', label)
    sarimax(data, label)
    print('started pure harmonic', label)
    pure_harmonic_regression(data, label)

In [68]:
for label in endog_week_dict_april:
    function_wrapper(endog_week_dict_april[label], label)

started sarima 0


started sarima 0
started sarimax 0


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 0
started sarima 1
started sarima 1
started sarimax 1


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

started pure harmonic 2
started sarima 3
started sarima 3
started sarimax 3
started pure harmonic 3
started sarima 4
started sarima 4
started sarimax 4


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 4
started sarima 5
started sarima 5
started sarimax 5


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 5


In [69]:
for label in endog_week_dict_april:
    print(f"{label} has the following sum {endog_week_dict_april[label].Count.sum()}")

0 has the following sum 205229
1 has the following sum 18348.0
2 has the following sum 3778.0
3 has the following sum 10914.0
4 has the following sum 168367
5 has the following sum 29411


In [70]:
metric_df = mlflow.search_runs(experiment_ids=experiment.experiment_id)

In [71]:
metric_df[(metric_df["params.label"] == '4') | (metric_df["params.label"] == '0')].sort_values(['metrics.RMSE_Test'])


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAE_Test,metrics.RMSE_Test,metrics.RMSE_Train,metrics.MAE_Test_Percentage,...,params.test_size,params.train_size,params.label,params.order,params.seasonal_order,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.user
4,8cc1df0b0caa4280951f7db6652d9d76,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:18:47.575000+00:00,2023-09-21 08:18:50.417000+00:00,159.847102,219.178388,119.210511,3.330148,...,48,480,4,None,None,"[{""run_id"": ""8cc1df0b0caa4280951f7db6652d9d76""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,april_harmonic_week_4,LOCAL,Max_G
5,f1da8ebf65ab43bc9d74971fccccfa04,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:18:32.518000+00:00,2023-09-21 08:18:47.562000+00:00,188.694218,242.935390,79.976244,3.931130,...,48,480,4,None,None,"[{""run_id"": ""f1da8ebf65ab43bc9d74971fccccfa04""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,SARIMAX 4,LOCAL,Max_G
7,0e4f04dc246f46928f0fa46cfed5e01f,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:18:20.346000+00:00,2023-09-21 08:18:28.765000+00:00,250.726695,313.184534,84.972098,5.223473,...,48,480,4,"(0, 1, 6)","(1, 1, 1, 24)","[{""run_id"": ""0e4f04dc246f46928f0fa46cfed5e01f""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(0, 1, 6), (1, 1, 1, 24), 4",LOCAL,Max_G
20,cb0a2f1aecc644a0b3ce87934e1233d1,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:16:39.641000+00:00,2023-09-21 08:16:42.532000+00:00,233.075016,315.766448,113.403676,4.855730,...,48,480,0,None,None,"[{""run_id"": ""cb0a2f1aecc644a0b3ce87934e1233d1""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,april_harmonic_week_0,LOCAL,Max_G
6,5063f22d9c194b23bd652e72d40db092,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:18:28.784000+00:00,2023-09-21 08:18:32.499000+00:00,303.068655,351.038202,114.276025,6.313930,...,48,480,4,"(1, 0, 0)","(1, 1, 0, 24)","[{""run_id"": ""5063f22d9c194b23bd652e72d40db092""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(1, 0, 0), (1, 1, 0, 24), 4",LOCAL,Max_G
21,2786c67dd175441d9643b6f552773d9b,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:16:25.236000+00:00,2023-09-21 08:16:39.629000+00:00,295.451877,391.207393,69.876896,6.155247,...,48,480,0,None,None,"[{""run_id"": ""2786c67dd175441d9643b6f552773d9b""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,SARIMAX 0,LOCAL,Max_G
22,2044ff058b694723a706a79bb2dbc8a1,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:16:21.462000+00:00,2023-09-21 08:16:25.219000+00:00,330.578756,429.482745,108.404048,6.887057,...,48,480,0,"(1, 0, 0)","(1, 1, 0, 24)","[{""run_id"": ""2044ff058b694723a706a79bb2dbc8a1""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(1, 0, 0), (1, 1, 0, 24), 0",LOCAL,Max_G
23,5af7cff97bdd4b34a1ba75e2edcd6cd8,630003381308202376,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:16:11.947000+00:00,2023-09-21 08:16:21.442000+00:00,346.486057,441.543143,74.000912,7.218460,...,48,480,0,"(0, 1, 6)","(1, 1, 1, 24)","[{""run_id"": ""5af7cff97bdd4b34a1ba75e2edcd6cd8""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(0, 1, 6), (1, 1, 1, 24), 0",LOCAL,Max_G


# Predicting for all data

In [72]:
tracking_uri = "file:///C:/Users/Max_G/OneDrive/IUBH/5. Semester/Model Engeneering/Public_Transport_Forecasting/models/Tracking"
mlflow.set_tracking_uri(tracking_uri)
# Create a named experiment
mlflow.set_experiment("predicting_per_cluster_all_data_new_1")
experiment = mlflow.get_experiment_by_name("predicting_per_cluster_all_data_new_1")

2023/09/21 10:19:23 INFO mlflow.tracking.fluent: Experiment with name 'predicting_per_cluster_all_data_new_1' does not exist. Creating a new experiment.


In [73]:
for label in endog_week_dict_all_data:
    function_wrapper(endog_week_dict_all_data[label], label)

started sarima 0
started sarima 0
started sarimax 0
started pure harmonic 0
started sarima 1
started sarima 1
started sarimax 1


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 1
started sarima 2
started sarima 2
started sarimax 2


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 2
started sarima 3
started sarima 3
started sarimax 3


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 3
started sarima 4
started sarima 4
started sarimax 4
started pure harmonic 4
started sarima 5
started sarima 5
started sarimax 5


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


started pure harmonic 5


In [74]:
metric_df_all_data = mlflow.search_runs(experiment_ids=experiment.experiment_id)
metric_df_all_data[(metric_df_all_data["params.label"] == '4') | (metric_df_all_data["params.label"] == '0')].sort_values(['metrics.RMSE_Test'])



,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAE_Test,metrics.RMSE_Test,metrics.RMSE_Train,metrics.MAE_Test_Percentage,...,params.test_size,params.train_size,params.label,params.order,params.seasonal_order,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.user
23,bbe86d230fef4e3692d8f2feef5b0aa7,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:19:23.865000+00:00,2023-09-21 08:20:07.302000+00:00,144.194473,165.580808,76.665325,3.004052,...,48,3095,0,"(0, 1, 6)","(1, 1, 1, 24)","[{""run_id"": ""bbe86d230fef4e3692d8f2feef5b0aa7""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(0, 1, 6), (1, 1, 1, 24), 0",LOCAL,Max_G
22,a3be8c3e2265476db9c13aa8ae1f18b0,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:20:07.407000+00:00,2023-09-21 08:20:15.814000+00:00,144.203086,166.184916,94.773909,3.004231,...,48,3095,0,"(1, 0, 0)","(1, 1, 0, 24)","[{""run_id"": ""a3be8c3e2265476db9c13aa8ae1f18b0""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(1, 0, 0), (1, 1, 0, 24), 0",LOCAL,Max_G
5,bcdbef5ab415468398e00333f73b09c4,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:28:29.109000+00:00,2023-09-21 08:28:54.519000+00:00,136.579771,172.280843,83.428269,2.845412,...,48,3095,4,None,None,"[{""run_id"": ""bcdbef5ab415468398e00333f73b09c4""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,SARIMAX 4,LOCAL,Max_G
4,a78e5e7a530c4e8db96f246aa70b5393,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:28:54.542000+00:00,2023-09-21 08:28:57.430000+00:00,115.147209,178.809905,140.716307,2.398900,...,48,3095,4,None,None,"[{""run_id"": ""a78e5e7a530c4e8db96f246aa70b5393""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,april_harmonic_week_4,LOCAL,Max_G
21,70e06b8d0df24f0ab129df6135171dac,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:20:15.885000+00:00,2023-09-21 08:20:45.856000+00:00,159.511459,213.281145,78.418287,3.323155,...,48,3095,0,None,None,"[{""run_id"": ""70e06b8d0df24f0ab129df6135171dac""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,SARIMAX 0,LOCAL,Max_G
20,595ff719f3ad485faaa4fb08584f3545,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:20:45.880000+00:00,2023-09-21 08:20:49.213000+00:00,185.866701,240.318412,159.612533,3.872223,...,48,3095,0,None,None,"[{""run_id"": ""595ff719f3ad485faaa4fb08584f3545""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,april_harmonic_week_0,LOCAL,Max_G
7,ad59ec9285584f9aa82527ca0c0ad481,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:27:34.157000+00:00,2023-09-21 08:28:17.781000+00:00,214.528913,240.362690,82.133156,4.469352,...,48,3095,4,"(0, 1, 6)","(1, 1, 1, 24)","[{""run_id"": ""ad59ec9285584f9aa82527ca0c0ad481""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(0, 1, 6), (1, 1, 1, 24), 4",LOCAL,Max_G
6,ff859a2fc96f4452897825725109ccd0,926935621880050686,FINISHED,file:///C:/Users/Max_G/OneDrive/IUBH/5. Semest...,2023-09-21 08:28:17.851000+00:00,2023-09-21 08:28:29.059000+00:00,190.766366,244.478298,105.995783,3.974299,...,48,3095,4,"(1, 0, 0)","(1, 1, 0, 24)","[{""run_id"": ""ff859a2fc96f4452897825725109ccd0""...",c:\Users\Max_G\anaconda3\lib\site-packages\ipy...,"SARIMA(1, 0, 0), (1, 1, 0, 24), 4",LOCAL,Max_G
